# Dreamer Database Update

This script updates the database with the results from dreamer, run on the Valencia HPC. Jupyter notebook used so tables can be rerun and checked etc. easily and interactively.

In [ ]:
import csv
import pandas as pd
import os
import re
import fnmatch

import sys
sys.path.append('../../src')

from yamlHandling import find_yaml_files #this function finds the yaml files in a directory.
from mysqlConnection import databaseConnector #this function permits connection to a mysql database using a CSV file containing details of the db connection.
from mysqlConnection import agentToDB #this function takes a dictionary and ingresses it into a table
from mysqlConnection import removePreviouslyRunInstances #this function takes a set of yaml files and task names and removes any that have already got results in the database.
from mysqlConnection import selectID #this function finds the integer ID for a table given a particular column name and value

## Database Connection

A function for connecting to the database.

In [ ]:
mycursor, connection = databaseConnector('../../scripts/databaseConnectionDetails.csv')

mycursor.close()

print("Connection checked and closed.")

### Create Agent Table


In [ ]:
rerunAgentTable = False

mycursor, connection = databaseConnector('../../scripts/databaseConnectionDetails.csv')

if rerunAgentTable:
    dropTable = "DROP TABLE IF EXISTS  dreameragents, dreameragentinstanceresults, dreameragentintrainstanceresults;"
    mycursor.execute(dropTable)
    
    sql = "CREATE TABLE `dreameragents` (`agentid` INT AUTO_INCREMENT PRIMARY KEY, `agent_tag` VARCHAR(300), `aai_seed` INT, `training_curriculum` VARCHAR(100), UNIQUE(agent_tag, training_curriculum));"
    mycursor.execute(sql)

mycursor.close()

In [ ]:
dreamer_bc_all_dict = {
    "training_curriculum" : "bc-all", #which training curriculum was the agent trained on?
    "agent_tag" : "dreamer-bc-all",
    "aai_seed" : 9999
}
dreamer_bc_opc_all_dict = {
    "training_curriculum" : "bc_opc-all", #which training curriculum was the agent trained on?
    "agent_tag" : "dreamer-bc_opc-all",
    "aai_seed" : 9999
}
dreamer_bc_opc_strat_dict = {
    "training_curriculum" : "bc_opc-strat", #which training curriculum was the agent trained on?
    "agent_tag" : "dreamer-bc_opc-strat",
    "aai_seed" : 9999
}
dreamer_bc_opc_opt_all_dict = {
    "training_curriculum" : "bc_opc_opt-all", #which training curriculum was the agent trained on?
    "agent_tag" : "dreamer-bc_opc_opt-all",
    "aai_seed" : 9999
}
dreamer_bc_opc_opt_strat_dict = {
    "training_curriculum" : "bc_opc_opt-strat", #which training curriculum was the agent trained on?
    "agent_tag" : "dreamer-bc_opc_opt-strat",
    "aai_seed" : 9999
}


In [ ]:
agent_dict_list = [dreamer_bc_all_dict, dreamer_bc_opc_all_dict, dreamer_bc_opc_strat_dict, dreamer_bc_opc_opt_all_dict, dreamer_bc_opc_opt_strat_dict]

In [ ]:
mycursor, connection = databaseConnector('../../scripts/databaseConnectionDetails.csv')

for agent in agent_dict_list:
    agentToDB(mycursor, agent, table_name = "dreameragents")

connection.commit()

mycursor.close()

## Build Results Tables

In [ ]:
mycursor, connection = databaseConnector('../../scripts/databaseConnectionDetails.csv')

rebuildInstanceResultsTables = False

if rebuildInstanceResultsTables:
    print("Rebuilding results tables, dropping if they already exist.")

    dropInstanceResultsTables = "DROP TABLE IF EXISTS dreameragentinstanceresults, dreameragentintrainstanceresults;"
    mycursor.execute(dropInstanceResultsTables)
    
    createInstanceTable = "CREATE TABLE dreameragentinstanceresults(instanceid INT NOT NULL, agentid INT NOT NULL, finalreward FLOAT(53), FOREIGN KEY (instanceid) REFERENCES instances(instanceid), FOREIGN KEY(agentid) REFERENCES dreameragents(agentid), PRIMARY KEY (instanceid, agentid));"
    mycursor.execute(createInstanceTable)

    createIntraInstanceTable = "CREATE TABLE dreameragentintrainstanceresults(instanceid INT NOT NULL, agentid INT NOT NULL, step INT NOT NULL, stepreward FLOAT(53), xvelocity FLOAT(32), yvelocity FLOAT(32), zvelocity FLOAT(32), xpos FLOAT(32), ypos FLOAT(32), zpos FLOAT(32), FOREIGN KEY (instanceid) REFERENCES instances(instanceid), FOREIGN KEY(agentid) REFERENCES dreameragents(agentid), PRIMARY KEY(instanceid, agentid, step));"
    mycursor.execute(createIntraInstanceTable)

    print("Tables: dreameragentinstanceresults and dreameragentintrainstanceresults have been successfully built.")

mycursor.close()

## Push results to database

In [ ]:
log_folder = '../logdir/clean-eval/'

folders = os.listdir(log_folder)

In [ ]:
def find_files(directory, extension = "*.yml"):
    names = []
    
    for root, dirnames, filenames in os.walk(directory):
        for filename in fnmatch.filter(filenames, extension):
            names.append(filename)
    
    return names

In [ ]:
mycursor, connection = databaseConnector('../../scripts/databaseConnectionDetails.csv')

for folder in folders:
    agent_name = re.sub("a[1-5]", "dreamer", folder)
    agent_name = re.sub("_basic_controls|_op_controlsp3|_op_controlsp2|_op_controls|_op_testsp2|_op_tests|p3", "", agent_name)
    print(agent_name)

    agentid = selectID(mycursor, id_name = "agentid", table_name = "dreameragents", WHERE_column = "agent_tag", WHERE_clause = agent_name)
    print(agentid)

    task_set = re.search("basic_controls|op_controls|op_tests", folder).group()

    print(task_set)

    task_names = find_files(os.path.join(log_folder, folder, task_set))

    task_names = sorted(task_names)

    print(f"Number of task names: {len(task_names)}")

    episode_results = find_files(os.path.join(log_folder, folder, "episodes"), "*.csv")

    print(f"Number of episode results: {len(episode_results)}")

    for episode in episode_results:
        adhoc_episode_id = int(re.search('[0-9]+', episode).group())
        #print(adhoc_episode_id)
        if adhoc_episode_id <= len(task_names): #some instances have multiple runs, only taking first
            episode_name = task_names[adhoc_episode_id-1]
            #print(episode_name)
            instanceid = selectID(mycursor, id_name = "instanceid", table_name = "instances", WHERE_column = "instancename", WHERE_clause = episode_name)
            
            select_existing_tasks = f"""SELECT COUNT(*) FROM instances INNER JOIN 
            dreameragentinstanceresults ON instances.instanceid = dreameragentinstanceresults.instanceid WHERE 
            dreameragentinstanceresults.instanceid = {instanceid} AND dreameragentinstanceresults.agentid = {agentid};"""

            mycursor.execute(select_existing_tasks)

            already_run = int(mycursor.fetchone()[0])

            if already_run == 0:

                with open(os.path.join(log_folder, folder, "episodes", episode)) as csvfile:
                    csvreader = csv.DictReader(csvfile)
                    for row in csvreader:
                        try:
                            intraInstanceQuery = f"""INSERT INTO dreameragentintrainstanceresults(instanceid, 
                            agentid, step, stepreward, xvelocity, yvelocity, zvelocity, xpos, ypos, zpos) 
                            VALUES ({instanceid}, {agentid}, {int(row[' step'])}, {float(row[' cumulative reward'])}, {float(row[' vx'])}, {float(row[' vy'])}, {float(row[' vz'])}, {float(row[' px'])}, {float(row[' py'])}, {float(row[' pz'])});"""
                            mycursor.execute(intraInstanceQuery)
                            #connection.commit()
            
                        except:
                            print(f"There's something wrong with this step. Here's the query {intraInstanceQuery}")
                            pass

                        if row[' done'] == " True":
                            try:
                                insertInstanceResults = f"INSERT INTO dreameragentinstanceresults(instanceid, agentid, finalreward) VALUES ({instanceid}, {agentid}, {float(row[' cumulative reward'])});"
                                mycursor.execute(insertInstanceResults)
                                connection.commit()
                            except:
                                print("It looks like this agent has already been tested on this instance.")
                               
                            
            else:
                print("This instance has already been run. Moving to next.")

                pass


        else:
            pass

mycursor.close()

In [ ]:
mycursor, connection = databaseConnector('../../scripts/databaseConnectionDetails.csv')

select_existing_tasks = f"""SELECT COUNT(*) FROM instances INNER JOIN 
            dreameragentinstanceresults ON instances.instanceid = dreameragentinstanceresults.instanceid WHERE 
            dreameragentinstanceresults.instanceid = 1 AND dreameragentinstanceresults.agentid = 1;"""

mycursor.execute(select_existing_tasks)

already_run = int(mycursor.fetchone()[0])
print(already_run)

mycursor.close()

In [ ]:
print(instanceid)
print(episode_name)